In [1]:
import torch
import numpy as np
from scipy.io import loadmat
import pandas as pd
import matplotlib.pyplot as plt
from scipy import signal
import nolds

In [2]:
data = loadmat("exampleEMGdata180trial_train.mat")
data.keys()

dict_keys(['__header__', '__version__', '__globals__', 'Fs', 'actualFs', 'dataChTimeTr', 'label_names', 'labels', 'numCh'])

In [3]:
test =  loadmat("exampleEMGdata120trial_test.mat")
test_trials = test['dataChTimeTr']
test_labels = test['labels'].tolist()

In [4]:
def lowpassFilter(data, sr, threshold):
    b,a = signal.butter(2, threshold/(sr/2),'low')
    return signal.filtfilt(b,a,data)

In [5]:
def WL(data):
    wl = np.sum(np.abs(np.diff(data)))
    return wl / len(data)

    

In [6]:
def MAV(data):
    return np.sum(np.abs(data))/len(data)

In [7]:
def SSC(data,threshold):
    res = 0
    for i in range(1, len(data)-1):
        curr = (data[i]-data[i-1]) * (data[i+1]-data[i])
        if curr >= threshold:
            res += 1
    return res

In [8]:
def ZC(data):
    """
    Counts how many times the signal crosses zero. 
    """
    res = 0
    for i in range(1, len(data)):
        curr = data[i] * data[i-1]
        if curr < 0:
            res += 1
    return res

In [9]:
def VAR(data):
    """
    Measures the spread of signal values around the mean. It reflects the level of muscle activation.
    """
    return np.var(data)

In [10]:
def SampEn(data, emb_dim=2, tolerance=None):
    if not tolerance:
        tolerance = 0.2 * np.std(data)
    sampen = nolds.sampen(data, emb_dim, tolerance)
    return sampen

In [12]:
trials = data['dataChTimeTr']
labels = data['labels'].tolist()
sr = data['Fs']

In [15]:
# Extract feature for each channel in each trial
def feature_extraction(trials, labels):
    features = []
    for i in range(len(trials[0][0])):
        tmp = []
        for j in range(len(trials)):
            slice = trials[j,:,i]
            slice = lowpassFilter(slice, sr, 20)
            wl = WL(slice)
            mav = MAV(slice)
            ssc = SSC(slice, 0.001)
            zc = ZC(slice)
            var = VAR(slice)
            sampen = SampEn(slice) 
            tmp.extend([wl, mav, ssc,zc,var,sampen])
        features.append(tmp + labels[i])
    return features

In [21]:
features = feature_extraction(test_trials, test_labels)

In [17]:
columns = [
    'ch1_wl', 'ch1_mav', 'ch1_ssc', 'ch1_zc', 'ch1_var', 'ch1_sampen',
    'ch2_wl', 'ch2_mav', 'ch2_ssc', 'ch2_zc', 'ch2_var', 'ch2_sampen',
    'ch3_wl', 'ch3_mav', 'ch3_ssc', 'ch3_zc', 'ch3_var', 'ch3_sampen',
    'ch4_wl', 'ch4_mav', 'ch4_ssc', 'ch4_zc', 'ch4_var', 'ch4_sampen',
    'Label'
]


In [22]:
df = pd.DataFrame(features, columns = columns)

In [23]:
df

,ch1_wl,ch1_mav,ch1_ssc,ch1_zc,ch1_var,ch1_sampen,ch2_wl,ch2_mav,ch2_ssc,ch2_zc,...,ch3_zc,ch3_var,ch3_sampen,ch4_wl,ch4_mav,ch4_ssc,ch4_zc,ch4_var,ch4_sampen,Label
0,96.668119,1575.822831,1337,17,2.424986e+06,0.164761,45.933040,1327.244413,1333,4,...,9,7.998548e+05,0.177612,43.956101,1351.105382,1343,6,7.869392e+05,0.132046,3
1,73.210873,1671.244951,1346,10,3.628343e+06,0.098008,56.558171,1495.993838,1346,10,...,4,4.301781e+05,0.203704,61.082708,1460.636351,1337,8,1.544767e+06,0.160525,1
2,97.739879,2010.543695,1344,14,7.643579e+06,0.074756,64.367819,1775.689144,1340,6,...,6,7.331818e+05,0.154065,74.140561,1555.595075,1337,11,2.169159e+06,0.168494,1
3,119.219655,1789.626642,1320,27,4.463486e+06,0.203431,117.978393,1637.135425,1323,25,...,16,1.507305e+06,0.278613,150.382309,1953.261060,1335,32,6.214804e+06,0.222343,2
4,112.667013,1647.448251,1345,22,3.816822e+06,0.178623,101.706936,1560.063574,1337,22,...,10,6.360401e+05,0.215790,122.889667,1652.410203,1335,26,2.887768e+06,0.206866,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,123.708258,1925.292546,1333,18,4.328550e+06,0.155979,96.689035,1618.952020,1331,18,...,11,1.312097e+06,0.197222,134.163326,1796.183886,1330,25,4.777069e+06,0.190086,2
116,88.118865,1673.219146,1341,12,2.624138e+06,0.179289,45.863043,1409.330731,1335,8,...,8,9.444016e+05,0.214918,60.482872,1458.322581,1342,8,1.229643e+06,0.201804,1
117,152.158324,2152.098274,1335,22,1.037643e+07,0.119118,111.298784,1652.101578,1335,21,...,14,3.927218e+06,0.122368,156.091935,1954.859814,1331,33,5.906301e+06,0.214320,2
118,89.583230,1733.852459,1339,14,5.117183e+06,0.082889,56.681246,1535.736240,1333,8,...,8,6.809224e+05,0.199350,48.392975,1442.923886,1335,4,1.501305e+06,0.104434,1


In [24]:
df.to_csv("test_feature_set.csv", index=False)